In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

In [ ]:
# Check Tensorflow version
print(f"Tensorflow version: {tf.__version__}")

Tensorflow version: 2.3.0


In [ ]:
# Check GPU availability
print("GPU", "is available" if tf.config.list_physical_devices("GPU") else "is not available")

GPU is available


In [ ]:
(X, y), (X_test, y_test) = fashion_mnist.load_data()
print(f"Train image's shape: {X.shape}")
print(f"Train label's shape: {y.shape}")

num_classes = len(np.unique(y))
num_classes

Train image's shape: (60000, 28, 28)
Train label's shape: (60000,)


10

In [ ]:
X = X.reshape((X.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))

In [ ]:
X.shape

(60000, 28, 28, 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,
                                                  y,
                                                  test_size = 0.2,
                                                  random_state = 42)

In [ ]:
X_train.shape

(48000, 28, 28, 1)

In [ ]:
y_train.shape

(48000,)

## Instantiate Dataset Class (BATCHES)
* Train Batch
* Validation Batch
* Test Batch

In [ ]:
train_batch = tf.data.Dataset.from_tensor_slices((X_train, y_train))
validation_batch = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_batch = tf.data.Dataset.from_tensor_slices((X_test, y_test))

### Visualizing Training Batch

In [ ]:
images = []
labels = []
for image, label in train_batch.take(len(train_batch)):
  images.append(image)
  labels.append(label)

plt.figure(figsize = (5, 5))
for i in range(25):
  plt.subplot(5, 5, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(images[i], cmap = "gray")
  plt.axis("off")
plt.show();

## Helpful Functions

In [ ]:
def augmentation(x, y):
  x = tf.image.resize_with_crop_or_pad(x, 28 + 7, 28 + 7)
  x = tf.image.random_crop(x, [28, 28, 1])
  x = tf.image.random_flip_left_right(x)
  x = tf.image.random_flip_up_down(x)

  return x, y

In [ ]:
def normalize(x, y):
  x = tf.image.per_image_standardization(x)

  return x, y

In [ ]:
train_batch = (train_batch.map(augmentation)
               .map(normalize)
               .shuffle(len(train_batch))
               .batch(32))

validation_batch = (validation_batch
                    .map(normalize)
                    .batch(32))

test_batch = (test_batch
              .map(normalize)
              .batch(32))

## Build Model

In [1]:
def build_model():

  model = tf.keras.Sequential([
          tf.keras.layers.Conv2D(32, (3, 3), padding = "same", activation = "relu", input_shape = (X_train.shape[1:])),
          tf.keras.layers.MaxPooling2D((2, 2)),
          tf.keras.layers.Dropout(0.2),  

          tf.keras.layers.Conv2D(64, (3, 3), padding = "same", activation = "relu"),
          tf.keras.layers.MaxPooling2D((2, 2)),
          tf.keras.layers.Dropout(0.2),

          tf.keras.layers.Conv2D(128, (3, 3), padding = "same", activation = "relu"),
          tf.keras.layers.MaxPooling2D((2, 2)),
          tf.keras.layers.Dropout(0.2),

          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(128, activation = "relu"),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(num_classes, activation = "softmax")                    
  ])

  return model

In [ ]:
network = build_model()

network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         7

In [ ]:
network.compile(
    optimizer = "adam",
    loss = "categorical_crossentropy",
    metrics = ["accuracy"]
)

In [ ]:
# Early Stopping Callback
callbacks = [tf.keras.callbacks.EarlyStopping(monitor = "val_accuracy",
                                              patience = 3)]

In [ ]:
history = network.fit(train_batch,
                      epochs = 20,
                      callbacks = callbacks,
                      validation_freq = 1,
                      validation_data = validation_batch)

Epoch 1/20
1500/1500 [==============================] - 13s 9ms/step - loss: 1.3388 - accuracy: 0.5601 - val_loss: 0.6682 - val_accuracy: 0.7449
Epoch 2/20
1500/1500 [==============================] - 14s 9ms/step - loss: 0.7958 - accuracy: 0.7100 - val_loss: 0.6246 - val_accuracy: 0.7622
Epoch 3/20
1500/1500 [==============================] - 13s 9ms/step - loss: 0.7030 - accuracy: 0.7445 - val_loss: 0.5622 - val_accuracy: 0.7928
Epoch 4/20
1500/1500 [==============================] - 13s 9ms/step - loss: 0.6431 - accuracy: 0.7652 - val_loss: 0.4683 - val_accuracy: 0.8240
Epoch 5/20
1500/1500 [==============================] - 13s 9ms/step - loss: 0.6128 - accuracy: 0.7767 - val_loss: 0.4718 - val_accuracy: 0.8230
Epoch 6/20
1500/1500 [==============================] - 13s 9ms/step - loss: 0.5940 - accuracy: 0.7831 - val_loss: 0.4374 - val_accuracy: 0.8339
Epoch 7/20
1500/1500 [==============================] - 13s 9ms/step - loss: 0.5764 - accuracy: 0.7894 - val_loss: 0.4387 - val_ac

In [ ]:
loss, acc = network.evaluate(test_batch)

print(f"Test accuracy of the network: {acc}")

313/313 [==============================] - 1s 5ms/step - loss: 0.4209 - accuracy: 0.8489
Test accuracy of the network: 0.8489000201225281


In [ ]:
predictions = network.predict(test_batch)
predictions.shape

(10000, 10)

In [ ]:
predictions[:1]

array([[3.0610014e-08, 6.4963892e-08, 1.9254910e-10, 8.6702734e-09,
        9.3453973e-09, 7.8572554e-04, 1.3811891e-08, 1.1445500e-01,
        2.7435082e-08, 8.8475907e-01]], dtype=float32)

In [ ]:
image_number = 7
actual_label = labels[image_number]
predicted_label = np.argmax(predictions[image_number])

actual_label, predicted_label

(<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)>,
 6)

In [ ]:
network.save("final_network.h5")

In [ ]:
loaded_model = tf.keras.models.load_model("final_network.h5")

In [ ]:
predictions = loaded_model.predict(test_batch)
predictions[:1]

array([[3.0610014e-08, 6.4963892e-08, 1.9254910e-10, 8.6702734e-09,
        9.3453973e-09, 7.8572554e-04, 1.3811891e-08, 1.1445500e-01,
        2.7435082e-08, 8.8475907e-01]], dtype=float32)

In [ ]:
image_number = 7
actual_label = y_test[image_number]
predicted_label = np.argmax(predictions[image_number])

actual_label, predicted_label

(6, 6)

In [ ]:
predictions[44]

array([4.8109025e-04, 1.1239673e-05, 1.1409402e-02, 2.9984368e-03,
       6.2270385e-01, 8.9887529e-08, 3.6223269e-01, 2.8714069e-11,
       1.6320543e-04, 9.1933492e-09], dtype=float32)